<a href="https://colab.research.google.com/github/naresh-FD/react-testgen-coverage/blob/main/react_testgen_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q "torch>=2.1" transformers datasets peft accelerate bitsandbytes trl



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 20.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!ls -lh /content/drive/MyDrive

total 50K
drwx------ 2 root root 4.0K Feb  8 04:20 'Colab Notebooks'
-rw------- 1 root root  179 Jul 10  2024 'Nareshkumar - Reactjs.gslides'
drwx------ 2 root root 4.0K May  2  2023 'Python Class'
drwx------ 2 root root 4.0K Apr 28  2023  Resume
-rw------- 1 root root  38K Feb  7 15:14  training.jsonl


In [4]:
cp /content/drive/MyDrive/training.jsonl /content/training.jsonl

In [5]:
!ls -lh /content
!head -n 2 /content/training.jsonl


total 48K
drwx------ 5 root root 4.0K Feb  9 13:42 drive
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
-rw------- 1 root root  38K Feb  9 13:42 training.jsonl
{"messages":[{"role":"system","content":"You are a React testing expert. You generate comprehensive Jest + React Testing Library test files for React components. Your tests are production-quality, use best practices, and achieve high code coverage."},{"role":"user","content":"Generate a comprehensive Jest + React Testing Library test file for the following React component.\nThe tests should achieve at least 50% code coverage.\n\n## Component: Button\nWrapper: forwardRef\n\n## Props\n- className: unknown [required]\n- variant: string [optional] (default: 'default')\n- size: string [optional] (default: 'default')\n- isLoading: boolean [optional] (default: false)\n- leftIcon: unknown [required]\n- rightIcon: unknown [required]\n- children: unknown [required]\n- disabled: unknown [required]\n- props: unknown [required]\n\n## B

In [6]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/training.jsonl"
)["train"]

print("Total examples:", len(dataset))
print(dataset[0]["messages"][1]["content"][:300])

Generating train split: 0 examples [00:00, ? examples/s]

Total examples: 5
Generate a comprehensive Jest + React Testing Library test file for the following React component.
The tests should achieve at least 50% code coverage.

## Component: Button
Wrapper: forwardRef

## Props
- className: unknown [required]
- variant: string [optional] (default: 'default')
- size: string


In [7]:
print(dataset.column_names)
print(dataset[0]["messages"][0])
print(dataset[0]["messages"][1])


['messages']
{'role': 'system', 'content': 'You are a React testing expert. You generate comprehensive Jest + React Testing Library test files for React components. Your tests are production-quality, use best practices, and achieve high code coverage.'}
{'role': 'user', 'content': 'Generate a comprehensive Jest + React Testing Library test file for the following React component.\nThe tests should achieve at least 50% code coverage.\n\n## Component: Button\nWrapper: forwardRef\n\n## Props\n- className: unknown [required]\n- variant: string [optional] (default: \'default\')\n- size: string [optional] (default: \'default\')\n- isLoading: boolean [optional] (default: false)\n- leftIcon: unknown [required]\n- rightIcon: unknown [required]\n- children: unknown [required]\n- disabled: unknown [required]\n- props: unknown [required]\n\n## Buttons\n- "unlabeled" (type: button)\n\n## Conditional Rendering\n- Condition: isLoading → renders JSX\n\n## Source Code\n```tsx\nimport { forwardRef, type 

In [8]:
def to_text(ex):
    # assumes messages = [{role, content}, {role, content}, ...]
    msgs = ex["messages"]
    # minimal format: user prompt -> assistant answer
    # if you have system+user+assistant, this still works well
    user_parts = [m["content"] for m in msgs if m["role"] in ["user", "system"]]
    assistant_parts = [m["content"] for m in msgs if m["role"] == "assistant"]

    prompt = "\n\n".join(user_parts).strip()
    completion = "\n\n".join(assistant_parts).strip()

    return {"text": f"{prompt}\n\n### RESPONSE:\n{completion}"}

dataset_text = dataset.map(to_text)
print(dataset_text[0]["text"][:600])


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

You are a React testing expert. You generate comprehensive Jest + React Testing Library test files for React components. Your tests are production-quality, use best practices, and achieve high code coverage.

Generate a comprehensive Jest + React Testing Library test file for the following React component.
The tests should achieve at least 50% code coverage.

## Component: Button
Wrapper: forwardRef

## Props
- className: unknown [required]
- variant: string [optional] (default: 'default')
- size: string [optional] (default: 'default')
- isLoading: boolean [optional] (default: false)
- leftIco


In [9]:
!pip install -q "torch>=2.1" transformers datasets peft accelerate bitsandbytes trl


In [10]:
BASE_MODEL = "Qwen/Qwen2.5-Coder-1.5B-Instruct"


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
)

lora = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
)

model = get_peft_model(model, lora)
model.print_trainable_parameters()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

trainable params: 4,358,144 || all params: 1,548,072,448 || trainable%: 0.2815


In [12]:
def tokenize(ex):
    out = tok(
        ex["text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_ds = dataset_text.map(tokenize, remove_columns=dataset_text.column_names)
print(train_ds[0].keys())


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=5,          # small dataset => more epochs ok
    logging_steps=5,
    save_steps=50,
    bf16=True, # Changed from fp16=True
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=args
)

trainer.train()

Truncating train dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
5,1.018575


TrainOutput(global_step=5, training_loss=1.0185750007629395, metrics={'train_runtime': 76.3383, 'train_samples_per_second': 0.327, 'train_steps_per_second': 0.065, 'total_flos': 201937728307200.0, 'train_loss': 1.0185750007629395})

In [14]:
model.save_pretrained("/content/react-testgen-lora")
tok.save_pretrained("/content/react-testgen-lora")
!ls -lh /content/react-testgen-lora


total 20M
-rw-r--r-- 1 root root 1014 Feb  9 13:45 adapter_config.json
-rw-r--r-- 1 root root 8.4M Feb  9 13:45 adapter_model.safetensors
-rw-r--r-- 1 root root 2.5K Feb  9 13:45 chat_template.jinja
-rw-r--r-- 1 root root 5.2K Feb  9 13:45 README.md
-rw-r--r-- 1 root root  664 Feb  9 13:45 tokenizer_config.json
-rw-r--r-- 1 root root  11M Feb  9 13:45 tokenizer.json


In [15]:
import torch

def generate(prompt: str):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=600,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    return tok.decode(out[0], skip_special_tokens=True)

test_prompt = dataset_text[0]["text"].split("### RESPONSE:")[0].strip() + "\n\n### RESPONSE:\n"
print(generate(test_prompt))


You are a React testing expert. You generate comprehensive Jest + React Testing Library test files for React components. Your tests are production-quality, use best practices, and achieve high code coverage.

Generate a comprehensive Jest + React Testing Library test file for the following React component.
The tests should achieve at least 50% code coverage.

## Component: Button
Wrapper: forwardRef

## Props
- className: unknown [required]
- variant: string [optional] (default: 'default')
- size: string [optional] (default: 'default')
- isLoading: boolean [optional] (default: false)
- leftIcon: unknown [required]
- rightIcon: unknown [required]
- children: unknown [required]
- disabled: unknown [required]
- props: unknown [required]

## Buttons
- "unlabeled" (type: button)

## Conditional Rendering
- Condition: isLoading → renders JSX

## Source Code
```tsx
import { forwardRef, type ButtonHTMLAttributes, type ReactNode } from 'react';
import { Loader2 } from 'lucide-react';
import { c

In [16]:
def to_text(ex):
    msgs = ex["messages"]
    # Combine system+user into prompt, assistant into completion
    prompt = "\n\n".join([m["content"] for m in msgs if m["role"] in ["system","user"]]).strip()
    completion = "\n\n".join([m["content"] for m in msgs if m["role"] == "assistant"]).strip()
    return {"text": f"{prompt}\n\n### RESPONSE:\n{completion}"}

dataset_text = dataset.map(to_text)
print(dataset_text[0]["text"][:600])


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

You are a React testing expert. You generate comprehensive Jest + React Testing Library test files for React components. Your tests are production-quality, use best practices, and achieve high code coverage.

Generate a comprehensive Jest + React Testing Library test file for the following React component.
The tests should achieve at least 50% code coverage.

## Component: Button
Wrapper: forwardRef

## Props
- className: unknown [required]
- variant: string [optional] (default: 'default')
- size: string [optional] (default: 'default')
- isLoading: boolean [optional] (default: false)
- leftIco


In [17]:
from datasets import load_dataset

# Load a sample dataset (e.g., a small subset of alpaca)
# Alpaca dataset structure: 'instruction', 'input', 'output'
dataset = load_dataset("tatsu-lab/alpaca", split="train[:50]") # Load first 50 examples

print("Total examples:", len(dataset))
# Print an example of the new dataset's content
print(dataset[0]["instruction"][:300])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Total examples: 50
Give three tips for staying healthy.


In [18]:
def to_text(ex):
    prompt_parts = []
    if ex["instruction"]:
        prompt_parts.append(ex["instruction"])
    if ex["input"]:
        prompt_parts.append(f"Input: {ex['input']}")

    prompt = "\n\n".join(prompt_parts).strip()
    completion = ex["output"].strip()

    return {"text": f"{prompt}\n\n### RESPONSE:\n{completion}"}

dataset_text = dataset.map(to_text)
print(dataset_text[0]["text"][:600])

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Give three tips for staying healthy.

### RESPONSE:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.


In [19]:
!pip install -q "torch>=2.1" transformers datasets peft accelerate bitsandbytes trl


In [20]:
def tokenize(ex):
    out = tok(
        ex["text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_ds = dataset_text.map(tokenize, remove_columns=dataset_text.column_names)
print(train_ds[0].keys())

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [21]:
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=5,          # small dataset => more epochs ok
    logging_steps=5,
    save_steps=50,
    bf16=True, # Changed from fp16=True
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=args
)

trainer.train()

Truncating train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Step,Training Loss
5,1.938854
10,2.059491
15,1.557828
20,1.605249
25,1.421520
30,1.456320
35,1.484687


TrainOutput(global_step=35, training_loss=1.6462785175868444, metrics={'train_runtime': 142.2474, 'train_samples_per_second': 1.758, 'train_steps_per_second': 0.246, 'total_flos': 190973140715520.0, 'train_loss': 1.6462785175868444})

In [22]:
model.save_pretrained("/content/react-testgen-lora")
tok.save_pretrained("/content/react-testgen-lora")
!ls -lh /content/react-testgen-lora

total 20M
-rw-r--r-- 1 root root 1014 Feb  9 13:49 adapter_config.json
-rw-r--r-- 1 root root 8.4M Feb  9 13:49 adapter_model.safetensors
-rw-r--r-- 1 root root 2.5K Feb  9 13:49 chat_template.jinja
-rw-r--r-- 1 root root 5.2K Feb  9 13:49 README.md
-rw-r--r-- 1 root root  664 Feb  9 13:49 tokenizer_config.json
-rw-r--r-- 1 root root  11M Feb  9 13:49 tokenizer.json


In [23]:
import torch

def generate(prompt: str):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=600,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    return tok.decode(out[0], skip_special_tokens=True)

test_prompt = dataset_text[0]["text"].split("### RESPONSE:")[0].strip() + "\n\n### RESPONSE:\n"
print(generate(test_prompt))

Give three tips for staying healthy.

### RESPONSE:
1. Eat a balanced diet of fruits, vegetables, lean proteins, and whole grains.
2. Exercise regularly to improve cardiovascular health, increase muscle strength, and burn calories.
3. Get enough sleep each night to support your body's natural healing process and help regulate mood and energy levels. ###


In [24]:
BASE_MODEL = "Qwen/Qwen2.5-Coder-1.5B-Instruct"


In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb,
    device_map="auto",
)

lora = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
)

model = get_peft_model(model, lora)
model.print_trainable_parameters()


Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

trainable params: 4,358,144 || all params: 1,548,072,448 || trainable%: 0.2815


In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/content/out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=5,
    logging_steps=5,
    save_steps=50,
    fp16=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=args,
)

trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


NotImplementedError: "_amp_foreach_non_finite_check_and_unscale_cuda" not implemented for 'BFloat16'

In [27]:
from datasets import load_dataset

# Make sure you've uploaded training.jsonl to Colab first
dataset = load_dataset("json", data_files="/content/training.jsonl", split="train")

print(f"✅ Loaded {len(dataset)} training examples")
print(f"Sample keys: {dataset[0].keys()}")
print(f"Messages in first example: {len(dataset[0]['messages'])}")


✅ Loaded 5 training examples
Sample keys: dict_keys(['messages'])
Messages in first example: 3


In [28]:
def format_chat(example):
    """Convert messages array to ChatML format string"""
    text = ""
    for msg in example["messages"]:
        if msg["role"] == "system":
            text += f"<|im_start|>system\n{msg['content']}<|im_end|>\n"
        elif msg["role"] == "user":
            text += f"<|im_start|>user\n{msg['content']}<|im_end|>\n"
        elif msg["role"] == "assistant":
            text += f"<|im_start|>assistant\n{msg['content']}<|im_end|>\n"
    return {"text": text}

formatted_dataset = dataset.map(format_chat)
print(f"✅ Formatted {len(formatted_dataset)} examples")
print(f"Sample text length: {len(formatted_dataset[0]['text'])} chars")

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

✅ Formatted 5 examples
Sample text length: 5837 chars


In [30]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/output",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    max_grad_norm=0.3,
    weight_decay=0.01,
)

print("✅ Training args configured")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Learning rate: {training_args.learning_rate}")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


✅ Training args configured
   Epochs: 4
   Batch size: 2
   Learning rate: 0.0002


In [39]:
from trl import SFTTrainer, SFTConfig

# Move all args into SFTConfig (replaces both TrainingArguments and SFTTrainer params)
sft_config = SFTConfig(
    output_dir="/content/output",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    bf16=True, # Changed from fp16=True
    optim="paged_adamw_8bit",
    report_to="none",
    max_grad_norm=0.3,
    weight_decay=0.01,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    args=sft_config
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


🚀 Starting training...


Step,Training Loss
5,1.106988


✅ Training complete!


In [41]:
 # Save LoRA adapter
trainer.save_model("/content/output/final")
tok.save_pretrained("/content/output/final")

# Merge LoRA weights back into base model
from peft import AutoPeftModelForCausalLM

merged_model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/output/final",
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained("/content/output/merged", safe_serialization=True)
tok.save_pretrained("/content/output/merged")

print("✅ Model merged and saved!")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Model merged and saved!


In [42]:
# Install llama.cpp converter
!pip install -q llama-cpp-python

# Clone llama.cpp for conversion script
!git clone --depth 1 https://github.com/ggerganov/llama.cpp /content/llama_cpp

# Install requirements
!pip install -q -r /content/llama_cpp/requirements.txt

# Convert to GGUF
!python /content/llama_cpp/convert_hf_to_gguf.py \
    /content/output/merged \
    --outfile /content/react-testgen.gguf \
    --outtype q4_K_M

import os
size_mb = os.path.getsize("/content/react-testgen.gguf") / (1024*1024)
print(f"✅ GGUF file created: {size_mb:.0f} MB")
print("📥 Download: Click Files panel → right-click react-testgen.gguf → Download")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 20.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
Cloning into '/content/llama_cpp'...
remote: Enumerating objects: 2482, done.
remote: Counting objects: 100% (2482/2482), done.
remote: Compressing objects: 100% (1954/1954), done.
remote: Total 2482 (delta 507), reused 1717 (delta 455), pack-reused 0 (from 0)
Receiving objects: 100% (2482/2482), 27.43 MiB | 18.89 MiB/s, done.
Resolving deltas: 100% (507/507), done.
Updating files: 100% (2217/2217), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━

FileNotFoundError: [Errno 2] No such file or directory: '/content/react-testgen.gguf'